In [1]:
import json
from typing import Optional
from dataclasses import dataclass, field
from pathlib import Path

import torch
import transformers
from peft import PeftModel
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    GenerationConfig, 
    HfArgumentParser, 
    BitsAndBytesConfig,
)
from tqdm import tqdm

/home/andre/.cache/pypoetry/virtualenvs/elleelleaime-N7sg6QK--py3.11/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/andre/.cache/pypoetry/virtualenvs/elleelleaime-N7sg6QK--py3.11/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# If you need to use a specific GPU, you can set it here
# if torch.cuda.is_available():
#     # Set GPU:1 as the device
#     torch.cuda.set_device(1)
#     print(f"Using GPU: {torch.cuda.current_device()}")
# else:
#     print("CUDA is not available.")

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-hf",
    torch_dtype=torch.float16,
    # load_in_8bit=True,
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0
    ),
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model = PeftModel.from_pretrained(
    model,
    '../repairllama-lora',
    torch_dtype=torch.float16,
)
model.config.pad_token = tokenizer.pad_token = tokenizer.unk_token
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32016, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Lin

In [5]:
# Bug 05

buggy_code5 = """
    public ValueMarker(double value, Paint paint, Stroke stroke, 
                       Paint outlinePaint, Stroke outlineStroke, float alpha) {
// buggy code
//        super(paint, stroke, paint, stroke, alpha);
        <FILL_ME>
        this.value = value;
    }
"""

In [6]:
inputs = tokenizer(buggy_code5, return_tensors="pt")
inputs_len = inputs["input_ids"].shape[1]
inputs_ids = inputs["input_ids"].to(device)

In [7]:
generation_config = GenerationConfig(
    num_beams=10,
    early_stopping=True,
)

outputs = model.generate(
    input_ids=inputs_ids,
    max_new_tokens=256,
    num_return_sequences=10,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    generation_config=generation_config,
)

In [8]:
output_ids = outputs[:, inputs_len:]
output_patch = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [9]:
for each in output_patch:
    print(each)
    print('-----------------')

super(paint, stroke, outlinePaint, outlineStroke, alpha);
</s>
-----------------
super(outlinePaint, outlineStroke, paint, stroke, alpha);
</s>
-----------------
super(paint, stroke, alpha);
</s>
-----------------
super(paint, stroke, paint, stroke, alpha);
</s>
-----------------
super(paint, stroke, paint, outlineStroke, alpha);
</s>
-----------------
super(paint, outlinePaint, stroke, outlineStroke, alpha);
</s>
-----------------
super(paint, outlineStroke, paint, outlineStroke, alpha);
</s>
-----------------
super(paint, stroke, null, null, alpha);
</s>
-----------------
super(paint, paint, stroke, stroke, alpha);
</s>
-----------------
super(paint, stroke, outlinePaint, outlineStroke, alpha);
</s>
-----------------
